In [4]:
import tensorflow as tf
from tensorflow import keras
import time
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, GlobalAveragePooling1D, BatchNormalization, Conv2D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sklearn
from scipy.spatial.distance import cdist, pdist
from scipy import stats
from sklearn.metrics import DistanceMetric
from tslearn.datasets import UCR_UEA_datasets
from tslearn.neighbors import  KNeighborsTimeSeries
from sklearn.metrics import accuracy_score
from scipy.interpolate import interp1d
from sklearn import preprocessing
print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False
# from counterfactual_utils import label_encoder, ucr_data_loader
%matplotlib inline

TF version:  2.16.1
Eager execution enabled:  False


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset

label_file = '../../data/raw/class_labels.csv'
data_dir = '../../data/raw/data/'

df_labels = pd.read_csv(label_file)
label_map = dict(zip(df_labels['File'], df_labels['Label']))

X = []
y = []
cols_to_keep = ['time_tag''p3_flux_ic']#,'p5_flux_ic', 'p7_flux_ic', 'long']
timestamps_list=[]
filenames=[]
for (fname,nr) in os.listdir(data_dir):
    if fname.endswith('.csv') and fname in label_map  :
        file_path = os.path.join(data_dir, fname)
        ts_df = pd.read_csv(file_path)

        ts_selected = ts_df.reindex(columns=cols_to_keep, fill_value=np.nan)
        ts_data = ts_selected.astype(float).values
        X.append(ts_selected)
        y.append(label_map[fname])
        filenames.append(fname)

# shape: (n_samples, n_timestamps, n_features)
X = to_time_series_dataset(X)  # will pad with NaN if needed
X[np.isnan(X)] = 0
y = np.array(y)
